In [93]:
from collections import namedtuple
from datetime import datetime
from collections import Counter


def get_clean_read(file):
    with open(file) as f:
        for line in f:
            yield line.strip('\n').split(',')


data_type = ('INT', 'STR', 'STR', 'STR', 'DATE', 'INT', 'STR', 'STR', 'STR')

_cast_data_to_type_key = {'DATE': lambda x: datetime.strptime(x, '%m/%d/%Y').date(), 'INT': lambda x: int(x)}


def cast_data_to_type(data_type, value):
    return _cast_data_to_type_key.get(data_type, lambda x: str(x))(value)


def cast_row(data_row, data_types=data_type):
    return [cast_data_to_type(dt, v) for dt, v in zip(data_types, data_row)]


def parking_ticket(file='nyc_parking_tickets_extract.csv'):
    data = get_clean_read(file)
    headers = [n.replace(' ', '_').lower() for n in next(data)]
    ParkingTicket = namedtuple('ParkingTicket', headers)
    for line in data:
        yield ParkingTicket(*cast_row(line))


def violations_by_car_mark(park_ticks=parking_ticket()):
    return Counter(ticket.vehicle_make for ticket in park_ticks)


In [70]:
nyc_park_tick = parking_ticket()
next(nyc_park_tick), next(nyc_park_tick), next(nyc_park_tick)

(ParkingTicket(summons_number=4006478550, plate_id='VAD7274', registration_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_body_type='4D', vehicle_make='BMW', violation_description='BUS LANE VIOLATION'),
 ParkingTicket(summons_number=4006462396, plate_id='22834JK', registration_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_body_type='VAN', vehicle_make='CHEVR', violation_description='BUS LANE VIOLATION'),
 ParkingTicket(summons_number=4007117810, plate_id='21791MG', registration_state='NY', plate_type='COM', issue_date=datetime.date(2017, 4, 10), violation_code=5, vehicle_body_type='VAN', vehicle_make='DODGE', violation_description='BUS LANE VIOLATION'))

In [94]:
violations_by_car_mark()

Counter({'TOYOT': 112,
         'HONDA': 106,
         'FORD': 104,
         'CHEVR': 76,
         'NISSA': 70,
         'DODGE': 45,
         'FRUEH': 44,
         'ME/BE': 38,
         'GMC': 35,
         'HYUND': 35,
         'BMW': 34,
         'LEXUS': 26,
         'INTER': 25,
         'JEEP': 22,
         'NS/OT': 18,
         'SUBAR': 18,
         'INFIN': 13,
         'LINCO': 12,
         'CHRYS': 12,
         'ACURA': 12,
         'AUDI': 12,
         'VOLVO': 12,
         'MITSU': 11,
         'ISUZU': 10,
         'CADIL': 9,
         'KIA': 8,
         'VOLKS': 8,
         'HIN': 6,
         'KENWO': 5,
         '': 5,
         'ROVER': 5,
         'BUICK': 5,
         'MAZDA': 5,
         'MERCU': 4,
         'JAGUA': 3,
         'SMART': 3,
         'PORSC': 3,
         'WORKH': 2,
         'SATUR': 2,
         'SCION': 2,
         'SAAB': 2,
         'HINO': 2,
         'FIR': 1,
         'OLDSM': 1,
         'PETER': 1,
         'CITRO': 1,
         'GEO': 1,
        

In [71]:
test = next(nyc_park_tick)
test.vehicle_make

'FORD'

In [72]:
def violations_by_car_mark(park_ticks):
    cars_violat = {}
    for tck in park_ticks:
        if tck.vehicle_make not in cars_violat.keys():
            cars_violat[tck.vehicle_make] = 1
        else:
            cars_violat[tck.vehicle_make] += 1
    return cars_violat

In [73]:
nyc_park_tick = parking_ticket()
violations_by_car_mark(nyc_park_tick)

{'BMW': 34,
 'CHEVR': 76,
 'DODGE': 45,
 'FORD': 104,
 'FRUEH': 44,
 'HONDA': 106,
 'LINCO': 12,
 'TOYOT': 112,
 'CADIL': 9,
 'CHRYS': 12,
 'FIR': 1,
 'GMC': 35,
 'HYUND': 35,
 'JAGUA': 3,
 'JEEP': 22,
 'LEXUS': 26,
 'ME/BE': 38,
 'MERCU': 4,
 'MITSU': 11,
 'NISSA': 70,
 'HIN': 6,
 'NS/OT': 18,
 'WORKH': 2,
 'ACURA': 12,
 'AUDI': 12,
 'INTER': 25,
 'ISUZU': 10,
 'KENWO': 5,
 'KIA': 8,
 'OLDSM': 1,
 'SUBAR': 18,
 'VOLVO': 12,
 'SATUR': 2,
 'SMART': 3,
 'INFIN': 13,
 'PETER': 1,
 '': 5,
 'CITRO': 1,
 'ROVER': 5,
 'BUICK': 5,
 'GEO': 1,
 'MAZDA': 5,
 'PORSC': 3,
 'VOLKS': 8,
 'YAMAH': 1,
 'BSA': 1,
 'MINI': 1,
 'PONTI': 1,
 'SPRI': 1,
 'PLYMO': 1,
 'SCION': 2,
 'UPS': 1,
 'FIAT': 1,
 'UD': 1,
 'UTILI': 1,
 'GMCQ': 1,
 'SAAB': 2,
 'HINO': 2,
 'STAR': 1,
 'AM/T': 1,
 'MI/F': 1}

In [74]:
from datetime import datetime


def get_clean_read(file):
    with open(file) as f:
        for line in f:
            yield line.strip('\n').split(',')


def cast_data_to_type(value):
    def try_cast():
        yield int, lambda x: int(x)
        yield datetime.date, lambda x: datetime.strptime(x, '%m/%d/%Y').date()
        yield str, lambda x: str(x)

    for expexted_type, cast_fn in try_cast():
        try:
            return cast_fn(value)
        except ValueError:
            continue


def cast_row(data_row):
    return [cast_data_to_type(v) for v in data_row]


def parking_ticket(file='nyc_parking_tickets_extract.csv'):
    data = get_clean_read(file)
    headers = [n.replace(' ', '_').lower() for n in next(data)]
    ParkingTicket = namedtuple('ParkingTicket', headers)
    for line in data:
        yield ParkingTicket(*cast_row(line))


In [75]:
p = parking_ticket()

In [76]:
next(p), next(p)

(ParkingTicket(summons_number=4006478550, plate_id='VAD7274', registration_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_body_type='4D', vehicle_make='BMW', violation_description='BUS LANE VIOLATION'),
 ParkingTicket(summons_number=4006462396, plate_id='22834JK', registration_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_body_type='VAN', vehicle_make='CHEVR', violation_description='BUS LANE VIOLATION'))

In [86]:
def violations_by_car_mark(park_ticks):
    cars_violat = {}
    for tck in park_ticks:
        cars_violat[tck.vehicle_make] = cars_violat.get(tck.vehicle_make, 0) + 1
    return cars_violat

In [90]:
from collections import Counter


def violations_by_car_mark(park_ticks):
    return Counter(tck.vehicle_make for tck in park_ticks)

In [91]:
nyc_park_tick = parking_ticket()
sorted(violations_by_car_mark(nyc_park_tick).items(), key=lambda x: x[1], reverse=True)

[('TOYOT', 112),
 ('HONDA', 106),
 ('FORD', 104),
 ('CHEVR', 76),
 ('NISSA', 70),
 ('DODGE', 45),
 ('FRUEH', 44),
 ('ME/BE', 38),
 ('GMC', 35),
 ('HYUND', 35),
 ('BMW', 34),
 ('LEXUS', 26),
 ('INTER', 25),
 ('JEEP', 22),
 ('NS/OT', 18),
 ('SUBAR', 18),
 ('INFIN', 13),
 ('LINCO', 12),
 ('CHRYS', 12),
 ('ACURA', 12),
 ('AUDI', 12),
 ('VOLVO', 12),
 ('MITSU', 11),
 ('ISUZU', 10),
 ('CADIL', 9),
 ('KIA', 8),
 ('VOLKS', 8),
 ('HIN', 6),
 ('KENWO', 5),
 ('', 5),
 ('ROVER', 5),
 ('BUICK', 5),
 ('MAZDA', 5),
 ('MERCU', 4),
 ('JAGUA', 3),
 ('SMART', 3),
 ('PORSC', 3),
 ('WORKH', 2),
 ('SATUR', 2),
 ('SCION', 2),
 ('SAAB', 2),
 ('HINO', 2),
 ('FIR', 1),
 ('OLDSM', 1),
 ('PETER', 1),
 ('CITRO', 1),
 ('GEO', 1),
 ('YAMAH', 1),
 ('BSA', 1),
 ('MINI', 1),
 ('PONTI', 1),
 ('SPRI', 1),
 ('PLYMO', 1),
 ('UPS', 1),
 ('FIAT', 1),
 ('UD', 1),
 ('UTILI', 1),
 ('GMCQ', 1),
 ('STAR', 1),
 ('AM/T', 1),
 ('MI/F', 1)]

In [ ]:
try_cast = [('DATE', lambda x: datetime.strptime(x, '%m/%d/%Y').date()), ('INT', lambda x: int(x)),
            ('STR', lambda x: str(x))]


def my_cast(value: str):
    for exp_type, cast_fn in try_cast():
        try:
            return cast_fn(value)
        except ValueError:
            continue